In [147]:
import pandas as pd
import json
import re

In [148]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

----

In [27]:
# read in reference
df = pd.read_csv('../reference.csv')

----

In [18]:
# this file was generated by get_geocoded_locations.py
location_dict = json.load(open("geocoded_location_dict.txt"))

In [19]:
# example usage
location_dict[' Iowa City, IA']

[41.6612561, -91.5299106, u'Iowa City', u'Iowa', u'us']

In [31]:
# add lat/lon columns to df
df['geocoded_latitude'] = "NA"
df['geocoded_longitude'] = "NA"
# add city/state/country columns to df
df['city'] = "NA"
df['state'] = "NA"
df['country'] = "NA"

In [32]:
# adding above features to df
for index,row in df.iterrows():
    location = row['location']
    if location in location_dict:
        latitude,longitude,city,state,country = location_dict[location]
        df.ix[index, 'geocoded_latitude'] = latitude
        df.ix[index, 'geocoded_longitude'] = longitude
        df.ix[index, 'city'] = city
        df.ix[index, 'state'] = state
        df.ix[index, 'country'] = country

In [123]:
# df.to_pickle('data_with_geo.pkl')
# df = pd.read_pickle('data_with_geo.pkl')

----

----

In [125]:
df[df['geocoded_latitude']=="NA"].shape

(6045, 13)

There are 6045 rows with no GPS coordinates

In [126]:
df[df['geocoded_latitude']=="NA"].head()

,id,description,duration,location,reported_at,shape,sighted_at,geocoded_latitude,geocoded_longitude,city,state,country,soln
5,5,Sheriff&apos;s office calls to rept. that depu...,30 min.,"Brunswick County, ND",19951024,unknown,19951025,NA,NA,NA,NA,NA,1
75,75,Woman &amp; 4 neighbors witness &quot;bright r...,NaN,"Volinda, CA",19951003,unknown,19951003,NA,NA,NA,NA,NA,1
171,171,Telephoned Report:Man stopped on highway to wa...,5 min.,"Lakeview, MI",19950103,light,19941229,NA,NA,NA,NA,NA,1
180,180,"Family members witness 5 lights in sky hover, ...",75 min.,"Elisabethtown, KY",19950212,unknown,19950212,NA,NA,NA,NA,NA,1
188,188,Man reports witnessing UFO two years prior. (...,NaN,"Reedborough, VT",19950210,unknown,19930101,NA,NA,NA,NA,NA,1


In [127]:
from uszipcode import ZipcodeSearchEngine

In [128]:
search = ZipcodeSearchEngine()

In [129]:
search.by_city_and_state(" Lakeview", "MI")

[{"City": "Lakeview", "Density": 55.398712233779094, "HouseOfUnits": 2203, "LandArea": 80.76, "Latitude": 43.444815999999996, "Longitude": -85.29293840000001, "NEBoundLatitude": 43.5544119, "NEBoundLongitude": -85.1614819, "Population": 4474, "SWBoundLatitude": 43.32216589999999, "SWBoungLongitude": -85.362562, "State": "MI", "TotalWages": 47904398.0, "WaterArea": 1.29, "Wealthy": 10707.286097451944, "Zipcode": "48850", "ZipcodeType": "Standard"}]

In [130]:
search.by_city_and_state("ci", "il")

[{"City": "Calumet City", "Density": 4991.4093959731545, "HouseOfUnits": 15708, "LandArea": 7.45, "Latitude": 41.6099709, "Longitude": -87.5445302, "NEBoundLatitude": 41.64260480000001, "NEBoundLongitude": -87.5246421, "Population": 37186, "SWBoundLatitude": 41.581531, "SWBoungLongitude": -87.578858, "State": "IL", "TotalWages": 489006357.0, "WaterArea": 0.12, "Wealthy": 13150.2812079815, "Zipcode": "60409", "ZipcodeType": "Standard"}]

In [131]:
line = "So. Cal., CA"

In [132]:
matchObj = re.match( r'(.*), (.*)', line, re.M|re.I)

In [133]:
matchObj.group(1)

'So. Cal.'

In [134]:
matchObj.group(2)

'CA'

### Go through all rows with no GPS coordinate and attempt to fix
Attempt to extract the city and state from location text and use uszipcode package to see if there are any misspellings and if so get the correct city and state

In [135]:
# add a flag for rows that have no solution
df['soln'] = 1

In [136]:
for index, row in df[df['geocoded_latitude']=="NA"].iterrows():
    line = row['location']
    idx = row['id']
    try:
        matchObj = re.match( r'(.*), (.*)', line, re.M|re.I)
        city = matchObj.group(1)
        state = matchObj.group(2)
    except:
        df.ix[index, 'soln'] = 0
        continue
    # shortest city name in U.S. is 3 letters
    if len(state) < 2 or len(city) < 3:
        continue
    try:
        # at this point if the iteration didnt fail then we have city and state
        res = search.by_city_and_state(city, state)[0]
        df.ix[idx, 'geocoded_latitude'] = res.Latitude
        df.ix[idx, 'geocoded_longitude'] = res.Longitude
        df.ix[index, 'city'] = res.City
        df.ix[index, 'state'] = res.State
        df.ix[index, 'country'] = "us"
    except:
        df.ix[index, 'soln'] = 0
        continue

In [137]:
df[df['geocoded_latitude']=="NA"].shape

(5168, 13)

Managed to fix 877 rows

----

In [138]:
df = df.drop(['soln'], axis=1)

In [139]:
# retain ONLY rows that have a geolocation
df = df[df['geocoded_latitude']!="NA"]

In [140]:
df.shape

(55882, 12)

### Retain ONLY US UFO sightings
Emailed professor why we are doing this (datasets and hypothesis support only US sightings) and received his approval to do this:

In [141]:
df = df[df['country'] == "us"]

In [142]:
df.shape

(38511, 12)

We keep 38.5k/55.9k rows

In [150]:
df.head()

,id,description,duration,location,reported_at,shape,sighted_at,geocoded_latitude,geocoded_longitude,city,state,country
0,0,"Man repts. witnessing &quot;flash, followed by...",NaN,"Iowa City, IA",19951009,unknown,19951009,41.6613,-91.5299,Iowa City,Iowa,us
1,1,"Man on Hwy 43 SW of Milwaukee sees large, bri...",2 min.,"Milwaukee, WI",19951011,unknown,19951010,43.035,-87.9225,Milwaukee,Wisconsin,us
2,2,Telephoned Report:CA woman visiting daughter w...,NaN,"Shelton, WA",19950103,unknown,19950101,47.2151,-123.101,Shelton,Washington,us
3,3,Man repts. son&apos;s bizarre sighting of smal...,2 min.,"Columbia, MO",19950510,unknown,19950510,38.9519,-92.3337,Columbia,Missouri,us
4,4,Anonymous caller repts. sighting 4 ufo&apos;s ...,NaN,"Seattle, WA",19950614,unknown,19950611,47.6038,-122.33,Seattle,Washington,us


----

#### Currently, states are both full form and abbreviations. The following converts any abbreviations to full form

In [164]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [156]:
for index,row in df.iterrows():
    state = row['state']
    if len(state) == 2:
        if state not in states:
            print "ERROR"
        else:
            df.ix[index, 'state'] = states[state]

In [165]:
df.state.unique()

array([u'Iowa', u'Wisconsin', u'Washington', u'Missouri', u'North Dakota',
       u'Nevada', u'Pennsylvania', u'Illinois', u'Massachusetts',
       u'Georgia', u'Florida', u'Virginia', u'New York', u'Maine',
       u'California', u'Oregon', u'Connecticut', u'Arizona', u'Alaska',
       u'New Jersey', u'Michigan', u'Idaho', u'Tennessee', u'Alabama',
       u'Ohio', u'Indiana', u'South Dakota', u'Kansas', u'Texas',
       u'North Carolina', u'South Carolina', u'Minnesota', u'Rhode Island',
       u'Utah', u'Colorado', u'Montana', u'Louisiana', 'Kentucky',
       'Vermont', 'Hawaii', u'New Mexico', u'West Virginia',
       u'New Hampshire', u'Oklahoma', u'Puerto Rico', u'Arkansas',
       u'Wyoming', u'Mississippi', u'Nebraska', u'Maryland',
       u'District of Columbia', u'Delaware',
       u'United States Virgin Islands'], dtype=object)

----

In [168]:
df.to_csv('reference_w_loc.csv', index=False)

In [163]:
# df.to_pickle('../pickle_files/reference_w_loc.pkl')
df = pd.read_pickle('../pickle_files/reference_w_loc.pkl')